#

This notebook demonstrates how to connect to the Style Guard WebSocket service and send text for style checking.

In [4]:
import json
import websockets
import asyncio
from IPython.display import display, Markdown

## Define WebSocket Connection Function

In [5]:
async def connect_to_style_guard(text_samples):
    uri = "ws://localhost:8000/api/logic-inspector"

    character_summaries = """
    Jax 
    Chapter: 1 
    Jax dies on Xylos 
    """
    location_summaries = """
    Xylos
    Chapter: 1
    Xylos is a planet with a red sun.
    """
    #
    async with websockets.connect(uri) as websocket:
        # Send initial style prompt
        await websocket.send(
            json.dumps(
                {
                    "character_summaries": character_summaries,
                    "location_summaries": location_summaries,
                }
            )
        )

        # Process each text sample
        for i, text in enumerate(text_samples):
            print(f"\n--- Sending Text Sample {i+1} ---")
            print(f"Text: {text[:100]}..." if len(text) > 100 else f"Text: {text}")

            # Send the text for checking
            await websocket.send(json.dumps({"text": text}))

            # Continue receiving messages until we get the "done" signal
            while True:
                response = await websocket.recv()
                response_data = json.loads(response)

                if response_data.get("status") == "done":
                    break

                print("\nResponse Received:")
                if "text_with_comments" in response_data:
                    display(
                        Markdown(
                            f"**Text with Comments:**\n{response_data['text_with_comments']}"
                        )
                    )
                else:
                    print(response_data)

            # Small delay between requests
            await asyncio.sleep(1)

## Define Sample Text and Style Guide

In [7]:
# Define some text samples for style checking
text_samples = [
    # "The product was developed by our team over a period of six months and it has been lauded by critics for its innovative features, user-friendly interface, and cutting-edge technology.",
    # "I believe that the implementation of the new system will significantly improve productivity, efficiency and customer satisfaction.",
    "Chapter 5: Jax breaks his leg on Earth"    
]

## Run the WebSocket Connection

In [8]:
# Run the websocket connection
try:
    await connect_to_style_guard(text_samples)
except Exception as e:
    print(f"Error connecting to websocket: {e}")


--- Sending Text Sample 1 ---
Text: Chapter 5: Jax breaks his leg on Earth

Response Received:
{'error': 'No text provided'}

Response Received:
{'original_text': 'Jax breaks his leg on Earth', 'comment': 'In Chapter 1, Jax dies on Xylos. He should not be appearing alive in Chapter 5.'}


## Alternative Approach with Manual Await (if needed)

In [ ]:
# If the await above doesn't work directly in Jupyter, use this approach
# asyncio.run(connect_to_style_guard(style_guide, text_samples))